In [27]:
import pymysql
from datetime import datetime
from elasticsearch import Elasticsearch
from tqdm.notebook import tqdm
import json
import os

In [3]:
def get_json(path):
    with open(path,"r",encoding="utf8") as f:
        text = f.read()
    text = text[2:]
    text = f"[{text}]"
    datas = json.loads(text)
    return datas

In [13]:
datas = []
for fld in list(filter(lambda o: int(o)>=20230414, os.listdir("./data_files"))):
    tmp = list(filter(lambda o: "jobplanet" in o, os.listdir(f"./data_files/{fld}")))
    if len(tmp) < 1:
        continue
    datas.extend(get_json(f"./data_files/{fld}/{tmp[0]}"))

In [14]:
datas = list(filter(lambda o: o['Data'] is not None, datas))

In [15]:
with open("./envs.json","r",encoding="utf8") as f:
    envs = json.load(f)

In [16]:
filtered_data = list(filter(lambda o: o['DataType'] == "jobplanet_statistic", datas))
filtered_data.extend(list(filter(lambda o: len(o['Data'])>0, list(filter(lambda o: o['DataType'] == "jobplanet_review", datas)))))

# connect ES

In [17]:
es = Elasticsearch(
    hosts=envs['es_host'],
    port=envs['es_port'],
    http_auth=(envs['es_user'],envs['es_password']),
    timeout=envs['es_timeout'],
    max_retries=envs['es_max_retries']
)

In [18]:
err_list = []
sucess_li = []
n = 0

In [19]:
for data in tqdm(datas, desc='엘라스틱 적재중'):
    try:
        es.index(index="source_data", document=data)
        sucess_li.append(data['BusinessNum'])
    except Exception as e:
        err_list.append((data, e))

엘라스틱 적재중:   0%|          | 0/3669 [00:00<?, ?it/s]

In [20]:
len(err_list)

56

In [21]:
es.search(
    index="source_data",
    size=0,
    query={
        "bool":{
            "must":[
                {"match":{"DataType":"naver_trend"}}
            ],
            "filter":[
                {"range":{"SearchDate":{"gte":"2023-04-01","format":"yyyy-MM-dd"}}}
            ]
        }
    },
    track_total_hits=True
)

{'took': 848,
 'timed_out': False,
 '_shards': {'total': 6, 'successful': 6, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 119851, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [22]:
# sucess_li_str = ", ".join([f"'{item}'" for item in sucess_li])    # 리스트 데이터를 string형식으로 변환
sucess_li = tuple(set(int(item) for item in sucess_li)) # set으로 중복값 제거 후 리스트 튜플로 변환
sucess_li

(1010617859,
 1018116099,
 1010686472,
 1018125832,
 1018139656,
 1010248204,
 1018123789,
 1018145293,
 1018117652,
 1010882072,
 1018130975,
 1018138657,
 1011015713,
 1018126883,
 1018111532,
 1018139184,
 1018141244,
 1018119742,
 1018147909,
 1018124361,
 1010938442,
 1018147404,
 1018136659,
 1018119252,
 1010938966,
 1018133079,
 1011141727,
 1018137696,
 1010659937,
 1018139748,
 1018122854,
 1018134118,
 1018145895,
 1018139753,
 1018119795,
 1010273396,
 1018146934,
 1018139767,
 1011132024,
 1018122873,
 1010793081,
 1018118781,
 1018117759,
 1018139786,
 1018137740,
 1018136718,
 1018119311,
 1018147985,
 1010770583,
 1018104477,
 1018104991,
 1018121379,
 1018137769,
 1010699434,
 1018148010,
 1018116783,
 1018118840,
 1018139321,
 1010980028,
 1018135742,
 1018138303,
 1018140361,
 1018142409,
 1010581714,
 1018143446,
 1010145495,
 1010260187,
 1010679516,
 1010244829,
 1018144479,
 1010663649,
 1018142433,
 1018100452,
 1018142443,
 1010909933,
 1010453230,
 1018145522,

# connect mysql

In [23]:
# file_name = 'jobplanet_statistic'
# file_name = 'jobplanet_review'
# file_name = 'jobplanet_premium'

file_name = datas[0]['DataType']

In [24]:
def get_mysql_conn(my_host, my_user, my_passwd, my_database, my_connect_timeout):
    conn = None
    try:
        conn = pymysql.connect(
            host=my_host,
            user=my_user,
            passwd=my_passwd,
            database=my_database,
            connect_timeout=my_connect_timeout,
        )
    except Exception as e:
        print(e)
    return conn

In [25]:
# conn = get_mysql_conn(
#     my_host=envs.get("db_host"),
#     my_user=envs.get("db_user"),
#     my_passwd=envs.get("db_password"),
#     my_database=envs.get("db_database"),
#     my_connect_timeout=envs.get("db_connect_timeout"),
# )

conn = get_mysql_conn(
    my_host=envs.get("db_host2"),
    my_user=envs.get("db_user2"),
    my_passwd=envs.get("db_password2"),
    my_database=envs.get("db_database2"),
    my_connect_timeout=envs.get("db_connect_timeout"),
)

In [28]:
data_status = datetime.now().strftime("%Y%m%d")
sql = f"UPDATE SOURCE_DATA_STATUS SET {file_name} = {data_status} WHERE BIZ_NO IN {sucess_li}"

In [29]:
try : 
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()
    cur.close()
    conn.close()
except : 
    pass